# Parsing and creating sample index.txt file for reference

In [3]:
import json

# Specify the path to your JSON file
json_file = "D:\\TTDS\\CW-3\\eurojobs.json"

# Open the output index.txt file in write mode
with open('index_cw3.txt', 'w', encoding='utf-8') as index_file:
    # Load the JSON data
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Iterate through each item in the JSON array
    for item in data:
        id = item.get('id', '')
        title = item.get('title', '')
        company = item.get('company', '')
        date_posted = item.get('date_posted', '')
        description = item.get('description', '')

        # Write the extracted information to the index.txt file
        index_file.write(f'ID: {id}\n')
        index_file.write(f'TITLE: {title}\n')
        index_file.write(f'COMPANY: {company}\n')
        index_file.write(f'DATE POSTED: {date_posted}\n')
        index_file.write(f'DESCRIPTION:\n{description}\n\n')

print('Data extracted and saved to index_cw3.txt.')

Data extracted and saved to index_cw3.txt.


# Parsing and creating index file for operations with preprocessing

In [152]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import string
import re
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')




from langdetect import detect
from langcodes import Language


def desp_preprocessing(text):
    if text !='':
        try:
            # Detect the language
            language_code = detect(text)

            if language_code !='No features in text.':
                # Get the full form of the detected language
                language_full_form = Language.get(language_code).language_name()
            else:
                language_full_form='unknown'
                
        except :
            language_full_form='unknown'
            print("Error:")
    else:
        return ['description empty']
    # Tokenization
    tokens = word_tokenize(text.lower())

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Split tokens containing hyphens into separate tokens
    split_tokens = []
    for token in tokens:
        if '-' in token:
            split_tokens.extend(token.split('-'))
        else:
            # Retain non-alphabetic characters as individual tokens
            split_tokens.extend(re.findall(r'\b\w+\b|[^a-zA-Z0-9_]', token))
    # Remove empty strings from the list of tokens
    split_tokens = [token for token in split_tokens if token.strip()]

    # Remove stopwords

    # List of languages for stopwords removal
    stopwords_languages = ['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']

    # Remove stopwords for that language
    if language_full_form.lower() in stopwords_languages:
        # Get stopwords for the current language
        stopwords_by_language = set(stopwords.words(language_full_form.lower()))

        # Remove stopwords
        split_tokens = [word for word in split_tokens if word.lower() not in stopwords_by_language]
    else:
        split_tokens =split_tokens 

    # Stemming

    # List of supported languages
    stemmed_languages = ["arabic","danish","dutch","english","finnish","french","german","hungarian","italian","norwegian","portuguese","romanian","russian","spanish","swedish"]

    # Perform stemming for that language
    if language_full_form.lower() in stemmed_languages:
        stemmer = SnowballStemmer(language_full_form.lower())
        for i, word in enumerate(split_tokens):
            split_tokens[i] = stemmer.stem(word)
            
    return split_tokens


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [153]:
def rest_preprocessing(text):
    if text !='':
        # Detect the language
        language_code = detect(text)

        if language_code !='No features in text.':
            # Get the full form of the detected language
            language_full_form = Language.get(language_code).language_name()
        else:
            language_full_form='unknown'
    else:
        return ['title empty']

    # Tokenization
    tokens = word_tokenize(text.lower())

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    split_tokens = tokens

    # Remove stopwords

    # List of languages for stopwords removal
    stopwords_languages = ['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']

    # Remove stopwords for that language
    if language_full_form.lower() in stopwords_languages:
        # Get stopwords for the current language
        stopwords_by_language = set(stopwords.words(language_full_form.lower()))

        # Remove stopwords
        split_tokens = [word for word in split_tokens if word.lower() not in stopwords_by_language]
    else:
        split_tokens = split_tokens 
        
    return split_tokens


In [155]:
from collections import defaultdict
import json

# Specify the path to your JSON file
json_file = "D:\\TTDS\\CW-3\\eurojobs.json"

# Initialize a dictionary to store term frequencies
term_freq = defaultdict(lambda: defaultdict(list))
doc_freq = defaultdict(int)


# Load JSON data
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Iterate through each job listing in the JSON data
for job_listing in data:
    doc_id = job_listing['id']
    title = job_listing.get('title', '')
    company = job_listing.get('company', '')
    date_posted = job_listing.get('date_posted', '')
    description = job_listing.get('description', '')
    print(doc_id)
    # Combine title, company, and date_posted for preprocessing
    combined_text = f"{title} {company}"
    
    # Preprocess except description
    combined_tokens = rest_preprocessing(combined_text)
    
    # Add date_posted without preprocessing
    combined_tokens.append(date_posted)

#     for term in combined_tokens:
#         term_freq[term][doc_id].append("title")
#         doc_freq[term] += 1
    
    # Count term frequencies within the document
    term_positions = {}
    for position, term in enumerate(combined_tokens):
        if term not in term_positions:
            term_positions[term] = []
        term_positions[term].append(position)
    
    # Update the term frequency and document frequency dictionaries
    for term, positions in term_positions.items():
        term_freq[term][doc_id] = positions
        doc_freq[term] += 1
    
    # Preprocess description
    description_tokens = desp_preprocessing(description)
    
#     for term in description_tokens:
#         term_freq[term][doc_id].append("description")
#         doc_freq[term] += 1

    for position, term in enumerate(description_tokens):
        if term not in term_positions:
            term_positions[term] = []
        term_positions[term].append(position)
    
    # Update the term frequency and document frequency dictionaries
    for term, positions in term_positions.items():
        term_freq[term][doc_id] = positions
        doc_freq[term] += 1

# # Write the results to the output file
# output_file = 'index_cw3.txt'

# with open(output_file, 'w', encoding='utf-8') as index_file:
#     for term, doc_positions in sorted(term_freq.items()):
#         index_file.write(f'{term}:{doc_freq[term]}\n')
#         for doc_id, positions in doc_positions.items():
#             index_file.write(f'\t{doc_id}: {",".join(map(str, positions))}\n')
            

# Write the results to the output file
output_file = 'indexcw3.txt'

with open(output_file, 'w', encoding='utf-8') as index_file:
    for term, doc_positions in sorted(term_freq.items()):
        index_file.write(f'{term}:{doc_freq[term]}\n')
        for doc_id, positions in doc_positions.items():
            index_file.write(f'\t{doc_id}: {",".join(map(str, positions))}\n')


print('Term frequencies and document frequencies written to index_cw3.txt.')


eurojobs-429957692
eurojobs-431120853
eurojobs-419095726
eurojobs-418659003
eurojobs-421755172
eurojobs-431120863
eurojobs-430449377
eurojobs-424056591
eurojobs-424056613
eurojobs-428951182
eurojobs-420703739
eurojobs-426780917
eurojobs-421319503
eurojobs-432000506
eurojobs-423896272
eurojobs-431995664
eurojobs-428950980
eurojobs-432001067
eurojobs-432016150
eurojobs-430341246
eurojobs-423896273
eurojobs-428486360
eurojobs-428559425
eurojobs-429665408
eurojobs-431818971
eurojobs-420270727
eurojobs-429664526
eurojobs-429664576
eurojobs-429664811
eurojobs-419095726
eurojobs-417921704
eurojobs-430185221
eurojobs-429664610
eurojobs-430185217
eurojobs-430185212
eurojobs-430185234
eurojobs-431534734
eurojobs-428079790
eurojobs-428079792
eurojobs-428079791
eurojobs-431722773
eurojobs-431550129
eurojobs-422231141
eurojobs-431579090
eurojobs-431553966
eurojobs-431564886
eurojobs-431985112
eurojobs-431575604
eurojobs-431985124
eurojobs-431985700
eurojobs-431989409
eurojobs-431990650
eurojobs-431

eurojobs-431896528
eurojobs-431896542
eurojobs-431896525
eurojobs-431896547
eurojobs-431896544
eurojobs-296822770
eurojobs-316899038
eurojobs-325405789
eurojobs-329175939
eurojobs-336100823
eurojobs-336777473
eurojobs-336810994
eurojobs-262956454
eurojobs-262956461
eurojobs-329175912
eurojobs-262956456
eurojobs-327795730
eurojobs-262956463
eurojobs-262956462
eurojobs-262956465
eurojobs-262956457
eurojobs-262956466
eurojobs-277030677
eurojobs-277030674
eurojobs-338505394
eurojobs-258791255
eurojobs-258791259
eurojobs-258791262
eurojobs-258791263
eurojobs-431896567
eurojobs-431896569
eurojobs-348528548
eurojobs-341265494
eurojobs-348543494
eurojobs-350063371
eurojobs-352184705
eurojobs-351325541
eurojobs-352168317
eurojobs-351450357
eurojobs-351456844
eurojobs-353382116
eurojobs-428491143
eurojobs-428491138
eurojobs-428491407
eurojobs-428491365
eurojobs-428491380
eurojobs-353390533
eurojobs-354155592
eurojobs-353402091
eurojobs-431896468
eurojobs-431896470
eurojobs-431896489
eurojobs-431

eurojobs-431896290
eurojobs-431896292
eurojobs-431896297
eurojobs-431896294
eurojobs-431896296
eurojobs-431896299
eurojobs-431896301
eurojobs-431896303
eurojobs-431896307
eurojobs-431896305
eurojobs-431896279
eurojobs-431896281
eurojobs-431896284
eurojobs-431896286
eurojobs-431896288
eurojobs-392908964
eurojobs-373647868
eurojobs-392908976
eurojobs-392908965
eurojobs-373512860
eurojobs-373608570
eurojobs-376995587
eurojobs-377003740
eurojobs-377514270
eurojobs-373670206
eurojobs-380263799
eurojobs-380260927
eurojobs-357049163
eurojobs-356215611
eurojobs-380278952
eurojobs-356216467
eurojobs-359908357
eurojobs-362133460
eurojobs-364018358
eurojobs-362133952
eurojobs-362867955
eurojobs-373416560
eurojobs-373454963
eurojobs-392908972
eurojobs-392908966
eurojobs-392908967
eurojobs-431896366
eurojobs-392908971
eurojobs-431896348
eurojobs-431896351
eurojobs-431896352
eurojobs-392908973
eurojobs-431896356
eurojobs-431896360
eurojobs-431896358
eurojobs-431896354
eurojobs-431896362
eurojobs-431

eurojobs-392909039
eurojobs-392909045
eurojobs-392909038
eurojobs-392909040
eurojobs-392909047
eurojobs-392909046
eurojobs-392909048
eurojobs-431896126
eurojobs-431896127
eurojobs-431896131
eurojobs-431896428
eurojobs-431896129
eurojobs-431896430
eurojobs-431896133
eurojobs-431896135
eurojobs-431896436
eurojobs-431896137
eurojobs-431896432
eurojobs-431896138
eurojobs-431896434
eurojobs-431896438
eurojobs-431896140
eurojobs-431896142
eurojobs-431896446
eurojobs-431896442
eurojobs-431896444
eurojobs-431896440
eurojobs-431896448
eurojobs-431896450
eurojobs-431896458
eurojobs-431896456
eurojobs-431896452
eurojobs-431896454
eurojobs-431896460
eurojobs-431896466
eurojobs-431896464
eurojobs-431896462
eurojobs-392909050
eurojobs-392909049
eurojobs-392909052
eurojobs-392909051
eurojobs-392909053
eurojobs-392909054
eurojobs-392909055
eurojobs-392909056
eurojobs-431896107
eurojobs-431896112
eurojobs-431896110
eurojobs-431896114
eurojobs-431896120
eurojobs-431896116
eurojobs-431896118
eurojobs-431

eurojobs-392909064
eurojobs-431895987
eurojobs-431895993
eurojobs-431895991
eurojobs-431895995
eurojobs-431895989
eurojobs-431895997
eurojobs-431896008
eurojobs-431896010
eurojobs-392909109
eurojobs-431896012
eurojobs-431896017
eurojobs-431896015
eurojobs-431896019
eurojobs-431896013
eurojobs-392909101
eurojobs-431896021
eurojobs-392909100
eurojobs-431896065
eurojobs-431896067
eurojobs-431896071
eurojobs-431896073
eurojobs-431896069
eurojobs-392909118
eurojobs-431896075
eurojobs-431896077
eurojobs-392909110
eurojobs-392909112
eurojobs-392909111
eurojobs-392909113
eurojobs-392909114
eurojobs-392909116
eurojobs-392909115
eurojobs-392909119
eurojobs-392909117
eurojobs-431896083
eurojobs-431896081
eurojobs-431896079
eurojobs-431896023
eurojobs-431896025
eurojobs-392909103
eurojobs-392909102
eurojobs-392909104
eurojobs-392909105
eurojobs-392909106
eurojobs-392909107
eurojobs-392909108
eurojobs-431895999
eurojobs-431896002
eurojobs-431896004
eurojobs-431896006
eurojobs-392909055
eurojobs-392

...

eurojobs-412605686
eurojobs-412605744
eurojobs-412605758
eurojobs-412604557
eurojobs-412604563
eurojobs-412604806
eurojobs-412604933
eurojobs-412604987
eurojobs-412604955
eurojobs-412605042
eurojobs-412605142
eurojobs-412605779
eurojobs-412605185
eurojobs-412605145
eurojobs-412605850
eurojobs-412912208
eurojobs-412912219
eurojobs-412912211
eurojobs-412912213
eurojobs-412912224
eurojobs-412912253
eurojobs-412912226
eurojobs-431877024
eurojobs-431877044
eurojobs-431877026
eurojobs-431877046
eurojobs-431871736
eurojobs-431871738
eurojobs-431871740
eurojobs-431871742
eurojobs-431871744
eurojobs-431871746
eurojobs-431871748
eurojobs-431871750
eurojobs-431871754
eurojobs-431871752
eurojobs-431871714
eurojobs-431871716
eurojobs-431871718
eurojobs-431871721
eurojobs-431871723
eurojobs-431871725
eurojobs-431871729
eurojobs-431871727
eurojobs-431871731
eurojobs-431871733
eurojobs-425589061
eurojobs-425589059
eurojobs-425589057
eurojobs-425589053
eurojobs-413733129
eurojobs-413733165
eurojobs-413

eurojobs-431871516
eurojobs-431871515
eurojobs-431871518
eurojobs-431871520
eurojobs-431871524
eurojobs-431871522
eurojobs-431871531
eurojobs-431871537
eurojobs-431871533
eurojobs-431871535
eurojobs-431871539
eurojobs-431871543
eurojobs-431871541
eurojobs-431871545
eurojobs-431876810
eurojobs-431871547
eurojobs-431876804
eurojobs-431876806
eurojobs-431876808
eurojobs-431876814
eurojobs-431876812
eurojobs-431876819
eurojobs-431876816
eurojobs-431876821
eurojobs-431876823
eurojobs-431871548
eurojobs-431876825
eurojobs-431876827
eurojobs-431876829
eurojobs-431876836
eurojobs-431876831
eurojobs-431876834
eurojobs-425588717
eurojobs-425588721
eurojobs-425588725
eurojobs-425588727
eurojobs-425588719
eurojobs-425588729
eurojobs-425588731
eurojobs-425588734
eurojobs-425588736
eurojobs-425588738
eurojobs-431876838
eurojobs-431876840
eurojobs-431876842
eurojobs-431876844
eurojobs-431871527
eurojobs-431871525
eurojobs-431871529
eurojobs-413733271
eurojobs-413733272
eurojobs-413733267
eurojobs-413

eurojobs-412912718
eurojobs-412912720
eurojobs-412912730
eurojobs-412912743
eurojobs-412912752
eurojobs-412912757
eurojobs-412911157
eurojobs-412912755
eurojobs-412912760
eurojobs-412911178
eurojobs-407153299
eurojobs-407242893
eurojobs-407225824
eurojobs-407248341
eurojobs-407249252
eurojobs-407943520
eurojobs-407943524
eurojobs-407943526
eurojobs-413733430
eurojobs-413733450
eurojobs-413733452
eurojobs-413733431
eurojobs-413733432
eurojobs-413733457
eurojobs-413733458
eurojobs-413733433
eurojobs-413733434
eurojobs-413733459
eurojobs-413733461
eurojobs-413733436
eurojobs-413733464
eurojobs-413733438
eurojobs-431876613
eurojobs-431876615
eurojobs-431876619
eurojobs-431876617
eurojobs-431876621
eurojobs-431876625
eurojobs-431876635
eurojobs-431876637
eurojobs-431876640
eurojobs-431876642
eurojobs-431876644
eurojobs-431876646
eurojobs-431876648
eurojobs-431876650
eurojobs-431876652
eurojobs-431876654
eurojobs-431876627
eurojobs-431876629
eurojobs-431876631
eurojobs-431876633
eurojobs-413

eurojobs-413733654
eurojobs-431876448
eurojobs-407943889
eurojobs-407943895
eurojobs-407943897
eurojobs-407943907
eurojobs-407943899
eurojobs-407943909
eurojobs-407943879
eurojobs-407943867
eurojobs-407943869
eurojobs-407943873
eurojobs-407943871
eurojobs-407943875
eurojobs-407943883
eurojobs-407943877
eurojobs-407943885
eurojobs-407943887
eurojobs-412911580
eurojobs-412913367
eurojobs-412911582
eurojobs-412911583
eurojobs-412911644
eurojobs-412911653
eurojobs-412911669
eurojobs-412911671
eurojobs-412911654
eurojobs-412911675
eurojobs-412911680
eurojobs-412911696
eurojobs-412911709
eurojobs-412911711
eurojobs-412911598
eurojobs-412911590
eurojobs-412911606
eurojobs-412911600
eurojobs-412911609
eurojobs-412911615
eurojobs-412911626
eurojobs-412913375
eurojobs-412913377
eurojobs-412913383
eurojobs-412913384
eurojobs-412913379
eurojobs-412913391
eurojobs-412913395
eurojobs-412913400
eurojobs-412913412
eurojobs-407943919
eurojobs-407943911
eurojobs-407943921
eurojobs-407943929
eurojobs-431

eurojobs-431876392
eurojobs-431876394
eurojobs-407944235
eurojobs-407944204
eurojobs-407944208
eurojobs-407944214
eurojobs-407944210
eurojobs-412913850
eurojobs-412913935
eurojobs-412913860
eurojobs-412913852
eurojobs-412913857
eurojobs-412913872
eurojobs-412913871
eurojobs-412913879
eurojobs-412913896
eurojobs-412913880
eurojobs-412913892
eurojobs-412913899
eurojobs-407944374
eurojobs-412913894
eurojobs-413733868
Term frequencies and document frequencies written to index_cw3.txt.


In [126]:
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']
